In [32]:
import pandas as pd
import torch
import os
import numpy as np

In [33]:
def load_data_from_csv(file_path):
    df = pd.read_csv(file_path)
    labels = df[df.columns[-1]]
    features = df.drop(df.columns[-1], axis=1).values
    labels = np.where(labels == 'true', 1, 0)

    X = torch.tensor(features, dtype=torch.float32)
    X = X.reshape(-1, 33, 3)
    y = torch.tensor(labels, dtype=torch.int64)

    return X, y

In [34]:
from google.colab import drive, files
import os

drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/Colab Notebooks/mobile-proj/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
filepaths = []
for filename in os.listdir(data_path):
    filepath = os.path.join(data_path, filename)
    if os.path.isfile(filepath) and filepath[-3:] =="csv":
        filepaths.append(filepath)



In [36]:
file = filepaths[0]
X, y = load_data_from_csv(file)

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
import torch.nn as nn
import torch.nn.functional as F

class PostureDetectionModel(nn.Module):
    def __init__(self):
        super(PostureDetectionModel, self).__init__()

        self.fc1 = nn.Linear(99, 64)
        self.dropout1 = nn.Dropout(0.5)
        # self.fc1 = nn.Linear(99, 128)
        # self.dropout1 = nn.Dropout(0.5)
        # self.fc2 = nn.Linear(128, 64)
        # self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        x = x.view(-1, 99)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        # x = F.relu(self.fc2(x))
        # x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)

        x = torch.sigmoid(self.fc4(x))

        return x

model = PostureDetectionModel()
print(model)

PostureDetectionModel(
  (fc1): Linear(in_features=99, out_features=64, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)


In [39]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [40]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)


In [41]:
num_epochs = 2

for epoch in range(num_epochs):
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels.unsqueeze(1).float())

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}')
        running_loss = 0.0

[1, 1] loss: 0.071
[1, 2] loss: 0.062
[1, 3] loss: 0.058
[1, 4] loss: 0.060
[1, 5] loss: 0.056
[1, 6] loss: 0.056
[1, 7] loss: 0.047
[1, 8] loss: 0.046
[1, 9] loss: 0.045
[1, 10] loss: 0.045
[1, 11] loss: 0.048
[1, 12] loss: 0.038
[1, 13] loss: 0.046
[1, 14] loss: 0.032
[1, 15] loss: 0.029
[1, 16] loss: 0.026
[1, 17] loss: 0.016
[1, 18] loss: 0.024
[1, 19] loss: 0.036
[1, 20] loss: 0.026
[1, 21] loss: 0.038
[1, 22] loss: 0.032
[1, 23] loss: 0.015
[1, 24] loss: 0.020
[1, 25] loss: 0.009
[1, 26] loss: 0.016
[1, 27] loss: 0.017
[1, 28] loss: 0.009
[1, 29] loss: 0.017
[1, 30] loss: 0.007
[1, 31] loss: 0.009
[1, 32] loss: 0.016
[1, 33] loss: 0.015
[1, 34] loss: 0.017
[1, 35] loss: 0.008
[1, 36] loss: 0.005
[1, 37] loss: 0.006
[1, 38] loss: 0.002
[1, 39] loss: 0.007
[1, 40] loss: 0.005
[1, 41] loss: 0.004
[1, 42] loss: 0.008
[1, 43] loss: 0.001
[1, 44] loss: 0.006
[2, 1] loss: 0.004
[2, 2] loss: 0.006
[2, 3] loss: 0.001
[2, 4] loss: 0.006
[2, 5] loss: 0.004
[2, 6] loss: 0.003
[2, 7] loss: 0.

In [42]:
def evaluate_model(model):
    total = 0
    correct = 0

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)

            predicted = outputs.round()
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1).float()).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [43]:
total = 0
correct = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)

        predicted = outputs.round()
        total += labels.size(0)
        correct += (predicted == labels.unsqueeze(1).float()).sum().item()

accuracy = 100 * correct / total

In [44]:
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)



In [45]:
quantized_model = quantized_model.cpu().eval()
quantized_model = torch.quantization.convert(quantized_model)

accuracy = evaluate_model(quantized_model)
print(f'Accuracy of the quantized model on the test dataset: {accuracy:.2f}%')


Accuracy of the quantized model on the test dataset: 100.00%


In [50]:
def save(model, filename):
    save_path = "/content/drive/MyDrive/Colab Notebooks/mobile-proj"
    path = os.path.join(save_path, filename)
    print(path)
    torch.save(model.state_dict(), path)

save(model, 'model_checkpoint.pth')
save(quantized_model, 'model_qint8.pth')

/content/drive/MyDrive/Colab Notebooks/mobile-proj/model_checkpoint.pth
/content/drive/MyDrive/Colab Notebooks/mobile-proj/model_qint8.pth
